# scikit-learnのトレーニング♨

## Kaggleチュートリアル 編 > タイタニック生存者予測

## [目次](TableOfContents.ipynb)

## 参考
開発基盤部会 Wiki
- Kaggle - Competitions - Getting Started - 開発基盤部会 Wiki  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?Kaggle%20-%20Competitions%20-%20Getting%20Started

## [環境準備](ScikitLearnTraining0.ipynb)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
proxies = {
    "http": os.getenv("HTTP_PROXY"),
    "https": os.getenv("HTTPS_PROXY")
}

## データの読み込み

In [ ]:
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
data = pd.read_csv(url)

## データの確認

### 概要確認

#### 列名一覧 

In [ ]:
data.columns.tolist()

#### 変数説明

| 変数 | 説明 |
| ---- | ---- |
| PassengerId | 乗客者ID |
| Survived | 生存状況（0＝死亡、1＝生存） |
| Pclass | 旅客クラス（1＝1等、2＝2等、3＝3等） |
| Name | 名前 |
| Sex | 性別（male＝男性、female＝女性） |
| Age | 年齢 |
| SibSp | 同乗している兄弟（Siblings）や配偶者（Spouses）の数 |
| Parch | 同乗している親（Parents）や子供（Children）の数 |
| Ticket | チケット番号 |
| Fare | 旅客運賃 |
| Cabin | 客室番号 |
| Embarked | 出港地（C＝Cherbourg、Q＝Queenstown、S＝Southampton） |

#### データの最初の数行を表示

In [ ]:
print(data.head())

#### データフレームの情報を表示

In [ ]:
print(data.info())

#### 基本的な統計量を表示

In [ ]:
print(data.describe())

#### 各列の欠損値の数を表示

In [ ]:
print(data.isnull().sum())

#### 数値データの分布を確認

In [ ]:
data.hist(bins=20, figsize=(14,10))
plt.show()

#### カテゴリカル変数の可能性のある列

In [ ]:
data.select_dtypes(include=['object', 'category']).columns.tolist()

Pclassは数値なので抜けたが分布から判断可能。また、Nameは非該当、Ticket, Cabinは要確認。

#### カテゴリカル変数の値のカウント

In [ ]:
categorical_columns = ['Pclass', 'Sex','Ticket', 'Cabin', 'Embarked']

for col in categorical_columns:
    print(f"\n{col} 列の値のカウント:")
    print(data[col].value_counts())

#### Ticket, Cabinとはなにか？

#### Ticket

In [ ]:
# Ticket列の統計情報を確認
print('\n', data['Ticket'].describe(include='all'))

# Ticket列のデータを表示
print('\n', data['Ticket'].head(10))  # 先頭10行を表示

# Ticket列の欠損値の数を確認
missing_cabin_count = data['Ticket'].isnull().sum()
print(f'\nTicket列の欠損値の数: {missing_cabin_count}')

# Ticket列の欠損値を削除しユニークな値の数を確認
ticket_unique = data['Ticket'].dropna().unique()

print(f'\nTicket列のユニークな値の数: {len(ticket_unique)}')

# 重複のないCabin列の値を表示
print('\nTicket列のユニークな値:' + str(ticket_unique))

#### Cabin
先頭が一文字がデッキを表している模様。

In [ ]:
# Cabin列の統計情報を確認
print('\n', data['Cabin'].describe(include='all'))

# Cabin列のデータを表示
print('\n', data['Cabin'].head(10))  # 先頭10行を表示

# Cabin列の欠損値の数を確認
missing_cabin_count = data['Cabin'].isnull().sum()
print(f'\nCabin列の欠損値の数: {missing_cabin_count}')

# Cabin列の欠損値を削除しユニークな値の数を確認
cabin_unique = data['Cabin'].dropna().unique()

print(f'\nCabin列のユニークな値の数: {len(cabin_unique)}')

# 重複のないCabin列の値を表示
print('\nCabin列のユニークな値:' + str(cabin_unique))

### 詳細確認
特徴量選択に向けての詳細確認。

#### カラーパレットの定義

In [ ]:
palette = sns.color_palette('Accent', 10)

#### カテゴリカルデータの頻度分析

In [ ]:
categorical_columns = ['Pclass', 'Sex', 'Embarked']
'''
for col in categorical_columns:
    sns.countplot(x=col, data=data)
    plt.title(f'Distribution of {col}')
    plt.show()
'''

# サブプロットの設定
fig, axes = plt.subplots(len(categorical_columns), 2, figsize=(14, 6 * len(categorical_columns)))

for i, col in enumerate(categorical_columns):
    # カテゴリのカウント
    counts = data[col].value_counts()
    # 色のリストを定義
    colors = palette[:len(counts)]
    
    # 棒グラフ
    axes[i, 0].bar(counts.index, counts.values, color=colors)
    axes[i, 0].set_title(f'Counts of {col}')
    axes[i, 0].set_xlabel(col)
    axes[i, 0].set_ylabel('Counts')
    
    # 円グラフ
    axes[i, 1].pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90, colors=colors)
    axes[i, 1].set_title(f'Distribution of {col}')
    axes[i, 1].axis('equal')  # 円を丸くするために等軸比を設定

# レイアウト調整
plt.tight_layout()
plt.show()

#### 相関行列をヒートマップで表示
PclassとFareの相関関係が高い。

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

#### 数値の説明変数と目的変数の関係の確認
ココのコードはGPTに生成させても使えなかったので個別に指示。

##### Survived

In [ ]:
# 目的変数のカウント
target_column = 'Survived'
target_counts = data[target_column].value_counts()

# サブプロットの作成
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 色のリストを定義
colors = ['red', 'blue']
    
# 円グラフの作成
axes[0].pie(target_counts, labels=target_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)
axes[0].set_title(f'Distribution of {target_column}')
axes[0].axis('equal')  # 円を丸くするために等軸比を設定

# 棒グラフの作成
axes[1].bar(target_counts.index, target_counts.values, color=colors)
axes[1].set_title(f'Counts of {target_column}')
axes[1].set_xlabel(target_column)
axes[1].set_ylabel('Counts')

# グラフの表示
plt.tight_layout()
plt.show()

##### Age
年齢

In [ ]:
# サブプロットを作成
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 全体の年齢分布のヒストグラム
sns.histplot(data['Age'].dropna(), kde=True, color='white', bins=20, ax=axes[0])

# 生存者と死亡者の年齢分布のヒストグラム
sns.histplot(data[data['Survived'] == 1]['Age'].dropna(), kde=True, color='blue', bins=20, ax=axes[1])
sns.histplot(data[data['Survived'] == 0]['Age'].dropna(), kde=True, color='red', bins=20, ax=axes[1])

# グラフの表示
plt.tight_layout()
plt.show()

##### SibSp
同乗している兄弟（Siblings）や配偶者（Spouses）の数

In [ ]:
# 説明変数のカウント
sibsp_counts = data['SibSp'].value_counts()

# 色のリストを定義
colors = palette[:len(sibsp_counts)]

# サブプロットを作成
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# SibSpの構成を示す円グラフ
axes[0].pie(sibsp_counts, labels=sibsp_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)

# SibSpとSurvivedの関連を分析した集合縦棒グラフ
sns.countplot(x='SibSp', hue='Survived', data=data, ax=axes[1], palette={0: 'red', 1: 'blue'})

# SibSpとSurvivedの関連を分析した積み上げ縦棒グラフ
count_data = data.groupby(['SibSp', 'Survived']).size().unstack().fillna(0)
count_data_ratio = count_data.div(count_data.sum(axis=1), axis=0)
count_data_ratio.plot(kind='bar', stacked=True, ax=axes[2], color=['red', 'blue'])

# グラフの表示
plt.tight_layout()
plt.show()

##### Parch
同乗している親（Parents）や子供（Children）の数

In [ ]:
# 説明変数のカウント
parch_counts = data['Parch'].value_counts()

# 色のリストを定義
colors = palette[:len(parch_counts)]

# サブプロットを作成
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Parchの構成を示す円グラフ
axes[0].pie(parch_counts, labels=parch_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)

# ParchとSurvivedの関連を分析した集合縦棒グラフ
sns.countplot(x='Parch', hue='Survived', data=data, ax=axes[1], palette={0: 'red', 1: 'blue'})

# ParchとSurvivedの関連を分析した積み上げ縦棒グラフ
count_data = data.groupby(['Parch', 'Survived']).size().unstack().fillna(0)
count_data_ratio = count_data.div(count_data.sum(axis=1), axis=0)
count_data_ratio.plot(kind='bar', stacked=True, ax=axes[2], color=['red', 'blue'])

# グラフの表示
plt.tight_layout()
plt.show()

##### Fare
最低の価格帯以外は生き残る確率が高い。高価格帯は顕著に高い。

###### 全体

In [ ]:
# サブプロットを作成
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 全体の旅客料金分布のヒストグラム
sns.histplot(data['Fare'].dropna(), kde=True, color='white', bins=20, ax=axes[0])

# 生存者と死亡者の旅客料金分布のヒストグラム
sns.histplot(data[data['Survived'] == 1]['Fare'].dropna(), kde=True, color='blue', bins=20, ax=axes[1])
sns.histplot(data[data['Survived'] == 0]['Fare'].dropna(), kde=True, color='red', bins=20, ax=axes[1])

# グラフの表示
plt.tight_layout()
plt.show()

###### 10ポンド以下のレコードを除外

In [ ]:
# サブプロットを作成
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 旅客料金が少ない旅客のデータを削除
threshold_fare = 10  # この値より少ない旅客のデータを削除（適宜調整）
temp = data[data['Fare'] >= threshold_fare]

# 全体の旅客料金分布のヒストグラム
sns.histplot(temp['Fare'].dropna(), kde=True, color='white', bins=20, ax=axes[0])

# 生存者と死亡者の旅客料金分布のヒストグラム
sns.histplot(temp[temp['Survived'] == 1]['Fare'].dropna(), kde=True, color='blue', bins=20, ax=axes[1])
sns.histplot(temp[temp['Survived'] == 0]['Fare'].dropna(), kde=True, color='red', bins=20, ax=axes[1])

# グラフの表示
plt.tight_layout()
plt.show()

#### カテゴリカル説明変数と目的変数の関係の確認

In [ ]:
categorical_columns = ['Pclass', 'Sex', 'Embarked']

for col in categorical_columns:
    sns.countplot(x=col, hue='Survived', data=data, palette={0: 'red', 1: 'blue'})
    plt.title(f'Relationship between {col} and Survived')
    plt.show()

#### その他の説明変数と目的変数の関係の確認

##### Cabin
先頭位置文字がデッキを表しているので、コレで分析してみる。

In [ ]:
# Cabinの先頭の文字（デッキ）を抽出する
data['Deck'] = data['Cabin'].str[0]

# 説明変数の欠損値を保管してカウント
data['Deck'].fillna('Z', inplace=True)
deck_counts = data['Deck'].value_counts()
sorted_list = deck_counts.index.tolist()

# サブプロットを作成
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Deckの構成を示す棒グラフ
sns.barplot(x=deck_counts.index, y=deck_counts.values, ax=axes[0])

# DeckとSurvivedの関連を分析した集合縦棒グラフ
sns.countplot(x='Deck', hue='Survived', data=data, ax=axes[1], palette={0: 'red', 1: 'blue'}, order=sorted_list)

# DeckとSurvivedの関連を分析した積み上げ縦棒グラフ
count_data = data.groupby(['Deck', 'Survived']).size().unstack().fillna(0)
count_data = count_data.reindex(sorted_list)
count_data_ratio = count_data.div(count_data.sum(axis=1), axis=0)
count_data_ratio.plot(kind='bar', stacked=True, ax=axes[2], color=['red', 'blue'])

# グラフの表示
plt.tight_layout()
plt.show()

## データの前処理

| 変数 | 説明 |
| ---- | ---- |
| PassengerId | 不要 |
| Survived | 目的変数 |
| Pclass | 富豪ほど生存 |
| Sex | 女性ほど生存 |
| Age | 20歳未満の生存率が高い、幼児は特に生存率が高い。 |
| SibSp | 独り身と大所帯の生存率が低い。 |
| Parch | 独り身と大所帯の生存率が低い。 |
| Ticket | 不要 |
| Fare | 富豪ほど生存（Pclassと高い相関関係） |
| Cabin | 不要 |
| Embarked | C=Cherbourg、Q＝Queenstown、S＝Southamptonのうち、Southamptonが優遇されているが、間接的な影響の可能性 |

In [ ]:
data.columns.tolist()

### 特徴量エンジニアリング

#### Deck
データ分析で追加したDeckはそのまま流用

#### FamilySize
FamilySizeを追加

In [ ]:
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

In [ ]:
# 説明変数のカウント
familysize_counts = data['FamilySize'].value_counts()

# 色のリストを定義
colors = palette[:len(familysize_counts)]

# サブプロットを作成
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# FamilySizeの構成を示す円グラフ
axes[0].pie(familysize_counts, labels=familysize_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)

# FamilySizeとSurvivedの関連を分析した集合縦棒グラフ
sns.countplot(x='FamilySize', hue='Survived', data=data, ax=axes[1], palette={0: 'red', 1: 'blue'})

# FamilySizeとSurvivedの関連を分析した積み上げ縦棒グラフ
count_data = data.groupby(['FamilySize', 'Survived']).size().unstack().fillna(0)
count_data_ratio = count_data.div(count_data.sum(axis=1), axis=0)
count_data_ratio.plot(kind='bar', stacked=True, ax=axes[2], color=['red', 'blue'])

# グラフの表示
plt.tight_layout()
plt.show()

#### TicketGroupSize
TicketGroupSizeを追加

In [ ]:
Ticket_Count = dict(data['Ticket'].value_counts())
data['TicketGroupSize'] = data['Ticket'].map(Ticket_Count)

In [ ]:
# 説明変数のカウント
tgsize_counts = data['TicketGroupSize'].value_counts()

# 色のリストを定義
colors = palette[:len(tgsize_counts)]

# サブプロットを作成
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# TicketGroupSizeの構成を示す円グラフ
axes[0].pie(tgsize_counts, labels=tgsize_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)

# TicketGroupSizeとSurvivedの関連を分析した集合縦棒グラフ
sns.countplot(x='TicketGroupSize', hue='Survived', data=data, ax=axes[1], palette={0: 'red', 1: 'blue'})

# TicketGroupSizeとSurvivedの関連を分析した積み上げ縦棒グラフ
count_data = data.groupby(['TicketGroupSize', 'Survived']).size().unstack().fillna(0)
count_data_ratio = count_data.div(count_data.sum(axis=1), axis=0)
count_data_ratio.plot(kind='bar', stacked=True, ax=axes[2], color=['red', 'blue'])

# グラフの表示
plt.tight_layout()
plt.show()

#### Title
名前から敬称を抽出

In [ ]:
data['Title'] = data['Name'].map(lambda x: x.split(', ')[1].split('. ')[0])
data['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer', inplace=True)
data['Title'].replace(['Don', 'Sir',  'the Countess', 'Lady', 'Dona'], 'Royalty', inplace=True)
data['Title'].replace(['Mme', 'Ms'], 'Mrs', inplace=True)
data['Title'].replace(['Mlle'], 'Miss', inplace=True)
data['Title'].replace(['Jonkheer'], 'Master', inplace=True)

In [ ]:
# 説明変数のカウント
title_counts = data['Title'].value_counts()
sorted_list = title_counts.index.tolist()

# 色のリストを定義
colors = palette[:len(title_counts)]

# サブプロットを作成
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Titleの構成を示す円グラフ
axes[0].pie(title_counts, labels=title_counts.index, autopct='%1.1f%%', startangle=90, colors=colors)

# TitleとSurvivedの関連を分析した集合縦棒グラフ
sns.countplot(x='Title', hue='Survived', data=data, ax=axes[1], palette={0: 'red', 1: 'blue'}, order=sorted_list)

# TitleとSurvivedの関連を分析した積み上げ縦棒グラフ
count_data = data.groupby(['Title', 'Survived']).size().unstack().fillna(0)
count_data = count_data.reindex(sorted_list)
count_data_ratio = count_data.div(count_data.sum(axis=1), axis=0)
count_data_ratio.plot(kind='bar', stacked=True, ax=axes[2], color=['red', 'blue'])

# グラフの表示
plt.tight_layout()
plt.show()

### 欠損値の処理とカテゴリカル変数のエンコーディング
- エンコーディング前に欠損値の処理が必要。
- エンコーディング結果を欠損値の処理で使いたい。

#### Embarkedの欠損値処理
最頻値で補完

In [ ]:
data["Embarked"].fillna(data['Embarked'].mode()[0], inplace=True)

#### カテゴリカル変数のエンコーディング

In [ ]:
data = pd.get_dummies(data, columns=['Sex', 'Deck', 'Embarked', 'Title'], drop_first=True)

#### Ageの欠損値処理
推定する

In [ ]:
#data["Age"].fillna(data["Age"].mean(), inplace=True) # 平均で補完

# 推定に使用する項目を指定
temp = data[['Age','Pclass','Sex_male','Parch','SibSp','Title_Miss','Title_Mr','Title_Mrs','Title_Officer','Title_Royalty']]

# ラベル特徴量をワンホットエンコーディング
temp = pd.get_dummies(temp)

# 学習データとテストデータに分離し、numpyに変換
known_age = temp[temp.Age.notnull()].values  
unknown_age = temp[temp.Age.isnull()].values

# 学習データをX, yに分離
X = known_age[:, 1:]  
y = known_age[:, 0]

# ランダムフォレストで推定モデルを構築
rfr = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)
rfr.fit(X, y)

# 推定モデルを使って、テストデータのAgeを予測し、補完
predictedAges = rfr.predict(unknown_age[:, 1::])
data.loc[(data.Age.isnull()), 'Age'] = predictedAges 

In [ ]:
print(data.isnull().sum())

### 必要な説明変数列を選択

In [ ]:
data.columns.tolist()

In [ ]:
xy = data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

### 結果の確認

In [ ]:
print(xy)

In [ ]:
# 特徴量とターゲット変数の分離
X = xy.drop('Survived', axis=1)
y = xy['Survived']

In [ ]:
# 訓練データとテストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# ロジスティック回帰モデルの訓練
#model = LogisticRegression()
# ランダムフォレストモデルの定義と訓練
#model = RandomForestClassifier(n_estimators=100, random_state=42)
# 勾配ブースティングモデルの定義と訓練
model = GradientBoostingClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# モデルの評価
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)